# Tiling - Predicting - Mosaicing

##### Before the data can be used in the model, they have to tiled to 256x256


https://towardsdatascience.com/efficiently-splitting-an-image-into-tiles-in-python-using-numpy-d1bf0dd7b6f7

TODO
1) Tif lesen, georeferenzierten Punkt oben links für späteres Nutzen erstellen 
2) JPEG erstellen
3) tiling
4) Klassifizierung 
5) wieder zusammensetzten der Tiles 
6) Georeferenzierung wieder hinzufügen (maybe 5?))

In [48]:
import os
import sys
from PIL import Image
from osgeo import gdal
import numpy as np
import rasterio
from osgeo import osr
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS
import glob
import re
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

import imageio

In [31]:
savedir = "E:/W_Katrin/Kigali/Raster/tiles"
filename = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.tif"

## 1) Create Point for Georeferencing [upper left corner]

In [107]:
# get coordinates from raster
src = gdal.Open(filename)

ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform() # ulx, uly = upper left x and y values

# get projection from raster
wkt = src.GetProjection()
src_crs = osr.SpatialReference(wkt).GetAttrValue('AUTHORITY',1)
print("SRC_CRS:", src_crs)

# create Point 
point = Point(ulx, uly)
print("Point:", point)

#create new GeoDataFrame to save the point in
newdata = gpd.GeoDataFrame()
newdata['geometry'] = None
newdata.loc[0, 'geometry'] = point

# Set the GeoDataFrame's coordinate system to the same as the src raster 
newdata.crs = CRS.from_epsg(src_crs)
print("GeoDataFrame CRS:", newdata.crs)

#save GeoDataFrame
outfp = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_ReferencePointUL.shp"
newdata.to_file(outfp)

SRC_CRS: 32736
Point: POINT (173723.36172185623 9789230.701895246)
GeoDataFrame CRS: epsg:32736


## 2) Convert input - file (Tif) to JPEG
#### using gdal translate
https://gis.stackexchange.com/questions/42584/how-to-call-gdal-translate-from-python-code

In [32]:
!gdal_translate -of PNG -b 1 -b 2 -b 3 -scale -co "PGW=YES" E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.tif E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB.png

Input file size is 13162, 8267
0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 6: driver PNG does not support creation option PGW
Warning 6: PNG driver doesn't support data type Float32. Only eight bit (Byte) and sixteen bit (UInt16) bands supported. Defaulting to Byte



In [33]:
dst = gdal.Open("E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB.png")
dst.GetMetadata()
!gdalinfo E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB.png

Driver: PNG/Portable Network Graphics
Files: E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB.png
       E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB.png.aux.xml
Size is 13162, 8267
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 36S",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 36S",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",33,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
         

## 3) TILING of input data
##### - Input raster has to be tiled to feed the model for prediction - 

In [34]:
# define tiling function 
def tiling(filename, savedir, target_size): 
    img = Image.open(filename)
    width, height = img.size
    start_pos = start_x, start_y = (0, 0)
    cropped_image_size = w, h = target_size
    frame_num = 1
    for col_i in range(0, width, w):
        for row_i in range(0, height, h):
            crop = img.crop((col_i, row_i, col_i + w, row_i + h))
            length = 5
            padding='0'
            save_to= os.path.join(savedir, "testing_{col}_{row}.png".format(col=format(col_i, padding + '>'+str(length)), row=format(row_i, padding + '>'+str(length))))
            crop.save(save_to)


In [35]:
# run tiling
import warnings
savedir = "E:/W_Katrin/Kigali/Raster/tiles_1"
filename = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB.png"

warnings.simplefilter('ignore', Image.DecompressionBombWarning)

tiling(filename, savedir, target_size = (256,256))

## 4) Predict and save predicted masks 

In [41]:
#Load the model
model_improved = keras.models.load_model("E:/W_Katrin/InnovationLab/17_model_improved")

In [42]:
# add here the folder path, where your  data is stored
directory = "E:/W_Katrin/Kigali/Raster/tiles_1"
files = glob.glob(os.path.join(directory,'*'))

pattern_img = '.*\.png'
# create a list of all images in that directory [after converting them to jpg with the code in Github README]
files_img = list(filter(re.compile(pattern_img).match,files))

files_img = sorted(files_img)

print('Image files: {}\n'.format(len(files_img)))

Image files: 1716



In [43]:
# create a dataframe of the satellite images and their respective mask 
df_files = pd.DataFrame(columns=['img'])
df_files['img'] = files_img

In [44]:
def make_test_gen(df_test,target_size,batch_size,seed):# this function generates augmented data[!not only training data!]
    # or better said: it generates a function which generates augmented data[generator function]
    
    # https://www.pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation/
    # generate batches of tensor image data with real-time data augmentation 
    # batches of original training data are randomly transformed and then used for training (not the original data!)
  test_gen = keras.preprocessing.image.ImageDataGenerator()
    
    # read data from dataframe and perform powerful on-the-fly image augmentation with ImageDataGenerator
  test_generator = test_gen.flow_from_dataframe(
          dataframe=df_test,        
          x_col='img',
          target_size=target_size,
          batch_size=batch_size,
          seed = seed,
          class_mode=None,
          color_mode='rgb',
          shuffle=False)
  return test_generator

In [45]:
test_image_gen = make_test_gen(df_files, batch_size = 1,target_size = (256, 256), seed = 0) #seed 0 to keep the Reihenfolge

Found 1716 validated image filenames.


In [46]:
# Generate output predictions for the input samples
mask_predicted = model_improved.predict(test_image_gen, verbose = 1)

1716/1716 [==============================] - 223s 129ms/step


In [ ]:
df_files["mask_file"] = mask_predicted

In [100]:
print(df_files)

ValueError: Shape of passed values is (5, 512, 256, 3), indices imply (10, 5)

In [49]:
#save predicted maps

savedir = "E:/W_Katrin/Kigali/Raster/tiles_1"
for i in range(len(mask_predicted)): 
    mask_pred = np.squeeze(mask_predicted[i])
    #print(mask_pred)
    #print(mask_pred.dtype)
    mask_pred = (255*(mask_pred - np.min(mask_pred))/np.ptp(mask_pred)).astype("uint8")  
    #break
    padding = '0'
    length = 4
    save_to = os.path.join(savedir, "testing_{x}.tif".format(x = format(i, padding + '>' + str(length))))
    imageio.imwrite(save_to, mask_pred)

In [ ]:
# define function to display the results

def predict_buildings(img, mask_pred):
    i=0
    img=img[i].astype(int)
    mask_pred=np.squeeze(mask_pred[i])
    mask_pred_rend=np.argmax(mask_pred, axis=-1)
    plt.figure(figsize=0.8*np.array([3*6.4, 4.8]))
    plt.subplot(131) # = plt.subplot(1,3,1), 1 row, 4 columns,  first subplot (from right to left, first top row)
    plt.imshow(img)
    plt.subplot(132)
    plt.imshow(mask_pred)
    plt.subplot(133)
    plt.imshow(mask_pred_rend)
    plt.show()
    
    return mask_pred, mask_pred_rend

In [ ]:
mask_predicted_, mask_predicted_rend_= predict_buildings(img_test, mask_predicted)

## 5) Mosaic - Put images back together

### 5.1) Mosaic preparation

In [80]:
# create a dataframe of the satellite image tiles and their respective masks
directory_img = 'E:/W_Katrin/Kigali/Raster/tiles_1'
files = glob.glob(os.path.join(directory_img,'*'))
pattern_img = '.*\.png$'
files_img = list(filter(re.compile(pattern_img).match,files))
files_img = sorted(files_img)

directory_mask = "E:/W_Katrin/Kigali/Raster/tiles_1"
files_ = glob.glob(os.path.join(directory_mask,'*'))
pattern_mask = '.*\.tif$'
files_mask = list(filter(re.compile(pattern_mask).match,files_))
files_mask = sorted(files_mask)

df_files = pd.DataFrame(columns=['img','mask', 'X', 'Y'])
df_files['img'] = files_img
df_files['mask'] = files_mask


# read the upper left corner coordinates from the img-filename and save it in X and Y

for i in range(df_files.shape[0]): 
    string = df_files["img"].iloc[i]
    string = string.split('/')[4].split('.')[0].split('_')
    x = string[2]
    y = string[3]
    df_files['X'].iloc[i] = x
    df_files['Y'].iloc[i] = y

df_files

,img,mask,X,Y
0,E:/W_Katrin/Kigali/Raster/tiles_1\testing_0000...,E:/W_Katrin/Kigali/Raster/tiles_1\testing_0000...,00000,00000
1,E:/W_Katrin/Kigali/Raster/tiles_1\testing_0000...,E:/W_Katrin/Kigali/Raster/tiles_1\testing_0001...,00000,00256
2,E:/W_Katrin/Kigali/Raster/tiles_1\testing_0000...,E:/W_Katrin/Kigali/Raster/tiles_1\testing_0002...,00000,00512
3,E:/W_Katrin/Kigali/Raster/tiles_1\testing_0000...,E:/W_Katrin/Kigali/Raster/tiles_1\testing_0003...,00000,00768
4,E:/W_Katrin/Kigali/Raster/tiles_1\testing_0000...,E:/W_Katrin/Kigali/Raster/tiles_1\testing_0004...,00000,01024
...,...,...,...,...
1711,E:/W_Katrin/Kigali/Raster/tiles_1\testing_1305...,E:/W_Katrin/Kigali/Raster/tiles_1\testing_1711...,13056,07168
1712,E:/W_Katrin/Kigali/Raster/tiles_1\testing_1305...,E:/W_Katrin/Kigali/Raster/tiles_1\testing_1712...,13056,07424
1713,E:/W_Katrin/Kigali/Raster/tiles_1\testing_1305...,E:/W_Katrin/Kigali/Raster/tiles_1\testing_1713...,13056,07680
1714,E:/W_Katrin/Kigali/Raster/tiles_1\testing_1305...,E:/W_Katrin/Kigali/Raster/tiles_1\testing_1714...,13056,07936


In [85]:
# create worldfiles for every mask tile to add georeferencing to them 

def create_worldfiles(dataframe, res, point_path):
    point = gpd.read_file(point_path)
    x_coord = point["geometry"].x[0]
    y_coord = point["geometry"].y[0]
    for i in range(len(dataframe)): 
        directory = dataframe["mask"].iloc[i].split('\\')[0]
        mask_str = dataframe["mask"].iloc[i].split('\\')[1].split('.')[0]
        f = open(os.path.join(directory, mask_str + '.tfw'), "w")
        f.write('{pixel_size}\n'.format(pixel_size = res)) #pixel size x
        f.write('0.0\n') #rotation
        f.write('0.0\n') #rotation
        f.write('{pixel_size}\n'.format(pixel_size = -res)) #pixel size y
        f.write('{easting}\n'.format(easting = (x_coord + (res*int(dataframe["X"].iloc[i]))))) #location x
        f.write('{northing}\n'.format(northing = (y_coord + (res*int(dataframe["Y"].iloc[i])))))#location y
        f.close()  

In [86]:
pt_file = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_ReferencePointUL.shp"
create_worldfiles(dataframe = df_files, res =  0.4963, point_path = pt_file)

### 5.2) Mosaicing

In [81]:
os.getcwd()
os.chdir('E:/W_Katrin/Kigali/Raster/tiles_1')
os.getcwd()

'E:\\W_Katrin\\Kigali\\Raster\\tiles_1'

In [88]:
# write all paths ofpredicted masks in txt file
files_list = glob.glob('*.tif')

with open('mask_files.txt', 'w') as f:
    for item in files_list:
        f.write("%s\n" % item)

##### First try: Build vrt and then gdal merge 

In [90]:
!gdalbuildvrt -input_file_list mask_files.txt merged.vrt

0...10...20...30...40...50...60...70...80...90...100 - done.


In [92]:
!python "E:\W_Katrin\miniconda3\envs\tue_dl2\Lib\site-packages\GDAL-3.4.1-py3.10-win-amd64.egg-info\scripts\gdal_merge.py" init 255 -o mask_predicted_mosaic_2.tif -of GTiff -v merged.vrt


Processing file     1 of     1,  0.000% completed in 0 minutes.
Filename: merged.vrt
File Size: 13312x8448x3
Pixel Size: 0.496300 x -0.496300
UL:(173723.113572,9793296.639645)   LR:(180329.859172,9789103.897245)
Copy 0,0,13312,8448 to 0,0,13312,8448.
Copy 0,0,13312,8448 to 0,0,13312,8448.
Copy 0,0,13312,8448 to 0,0,13312,8448.


ERROR 4: init: No such file or directory
ERROR 4: 255: No such file or directory


In [93]:
# for better comparison try out with png (original ratser image tiles)

files_list = glob.glob('*.png')

with open('img_files.txt', 'w') as f:
    for item in files_list:
        f.write("%s\n" % item)

In [94]:
!gdalbuildvrt -input_file_list img_files.txt img_merged.vrt

0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 6: gdalbuildvrt does not support ungeoreferenced image. Skipping testing_00000_00000.png
Warning 6: gdalbuildvrt does not support ungeoreferenced image. Skipping testing_00000_00256.png
Warning 6: gdalbuildvrt does not support ungeoreferenced image. Skipping testing_00000_00512.png
Warning 6: gdalbuildvrt does not support ungeoreferenced image. Skipping testing_00000_00768.png
Warning 6: gdalbuildvrt does not support ungeoreferenced image. Skipping testing_00000_01024.png
Warning 6: gdalbuildvrt does not support ungeoreferenced image. Skipping testing_00000_01280.png
Warning 6: gdalbuildvrt does not support ungeoreferenced image. Skipping testing_00000_01536.png
Warning 6: gdalbuildvrt does not support ungeoreferenced image. Skipping testing_00000_01792.png
Warning 6: gdalbuildvrt does not support ungeoreferenced image. Skipping testing_00000_02048.png
Warning 6: gdalbuildvrt does not support ungeoreferenced image. Skipping testing_00000_02304.png
Warning 6: gdalbuildvrt does n

In [ ]:
!python "E:\W_Katrin\miniconda3\envs\tue_dl2\Lib\site-packages\GDAL-3.4.1-py3.10-win-amd64.egg-info\scripts\gdal_merge.py" init 255 -o mask_predicted_mosaic_2.tif -of GTiff -v merged.vrt

In [ ]:
gdalwarp -t_srs EPSG:32736

In [76]:
#with gdal merge? !no seperate!
import subprocess
files_list = glob.glob('*.tif')
#files_list = files_list[:2]



#print(files_list)

files_string = " ".join(files_list)
gdal_merge_loc = "E:\W_Katrin\miniconda3\envs\tue_dl2\Lib\site-packages\GDAL-3.4.1-py3.10-win-amd64.egg-info\scripts\gdal_merge.py"
#cmd = python + gdal_merge_loc + "-init 255 -o mask_predicted_mosaic_2.jpg -of JPEG -v" + files_string

#os.system(cmd)

In [91]:
!gdal_merge.py -o mask_predicted_mosaic_1.tif -of GTiff merged.vrt

Traceback (most recent call last):
  File "E:\W_Katrin\miniconda3\envs\tue_dl2\Scripts\gdal_merge.py", line 4, in <module>
    __import__('pkg_resources').run_script('GDAL==3.4.1', 'gdal_merge.py')
  File "C:\Python27\ArcGIS10.6\lib\site-packages\pkg_resources\__init__.py", line 3017, in <module>
    @_call_aside
  File "C:\Python27\ArcGIS10.6\lib\site-packages\pkg_resources\__init__.py", line 3003, in _call_aside
    f(*args, **kwargs)
  File "C:\Python27\ArcGIS10.6\lib\site-packages\pkg_resources\__init__.py", line 3030, in _initialize_master_working_set
    working_set = WorkingSet._build_master()
  File "C:\Python27\ArcGIS10.6\lib\site-packages\pkg_resources\__init__.py", line 659, in _build_master
    ws.require(__requires__)
  File "C:\Python27\ArcGIS10.6\lib\site-packages\pkg_resources\__init__.py", line 967, in require
    needed = self.resolve(parse_requirements(requirements))
  File "C:\Python27\ArcGIS10.6\lib\site-packages\pkg_resources\__init__.py", line 853, in resolve
   

In [ ]:
!gdalbuildvrt -input_file_list mask_files.txt merged.vrt